# AADT Confidence Interval - Interstate 99

## FHWA Links
* Guidelines for Obtaining AADT Estimates from Non-Traditional Sources:
    * https://www.fhwa.dot.gov/policyinformation/travel_monitoring/pubs/aadtnt/Guidelines_for_AADT_Estimates_Final.pdf
  
  
## AADT Analysis Locations
* 10 locations were used in the analysis
* Locations were determined based on the location on installed & recording Traffic Operations cameras
    * for additional information contact Zhenyu Zhu with Traffic Operations

## Traffic Census Data
* https://dot.ca.gov/programs/traffic-operations/census/traffic-volumes
* Back AADT, Peak Month, and Peak Hour usually represents traffic South or West of the count location.  
* Ahead AADT, Peak Month, and Peak Hour usually represents traffic North or East of the count location. Listing of routes with their designated  

* Because the Back & Ahead counts are included at each location in the Traffic Census Data, (e.g., "IRWINDALE, ARROW HIGHWAY") only one [OBJECTID*] per location was pulled; for this analysis the North Bound Nodes were used for the analysis. 
    * for more information see the diagram: https://traffic.onramp.dot.ca.gov/downloads/traffic/files/performance/census/Back_and_Ahead_Leg_Traffic_Count_Diagram.pdf

## StreetLight Analysis Data
* StreetLight Locations on Interstate 99 are one-direction, each location will contain two points: northbound and southbound
    * Analysis Type == Network Performance
    * Segment Metrics
    * 2022 was used to match currently available Traffic Census Data (as of 8/27/2025)
    * pulled a variety of Day Types, but plan to just look at """All Day Types"""
    * pulled a variety of Day Parts, but plan to just look at """All Day Parts"""




In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

### Pull in the Location Dictionaries

In [2]:
# pull in the coordinates from the utils docs
#from osow_frp_o_d_utils_v3 import origin_intersections, destination_intersections
from sr99_tc_locations_utils import sr_99_d3_tc_aadt_locations

### Identify the Google Cloud Storage path

In [3]:
# Identify the GCS path to the data
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/compare_traffic_counts/sr99_d3/"

### Pull in the Data

In [4]:
# This function will pull in the data and clean the column headers in a way that will make them easier to work with
def getdata_and_cleanheaders(path):
    # Read the CSV file
    df = pd.read_csv(path)

    # Clean column headers: remove spaces, convert to lowercase, and strip trailing asterisks
    cleaned_columns = []
    for column in df.columns:
        cleaned_column = column.replace(" ", "").lower().rstrip("*")
        cleaned_columns.append(cleaned_column)

    df.columns = cleaned_columns
    return df

In [5]:
# pull in the data & create dataframes
df_tc = getdata_and_cleanheaders(f"{gcs_path}caltrans_traffic_census_2022.csv")  # Traffic Census
df_stl = getdata_and_cleanheaders(f"{gcs_path}streetlight_sr99_d3_all_vehicles_2022_np.csv")  # StreetLight

In [6]:
# comparing
df_tc.to_csv("df_tc.csv", index=False)

In [7]:
# comparing
df_stl.to_csv("df_stl.csv", index=False)

In [8]:
def extract_aadt_comparison_data(df_stl, df_tc, mapping_dicts):
    """
    Extracts AADT counts from StreetLight and Traffic Census datasets 
    based on a mapping of zonename keys and objectid.

    Parameters:
    - df_stl (pd.DataFrame): StreetLight data with 'zonename' and 'averagedailysegmenttraffic(stlvolume)' columns.
    - df_tc (pd.DataFrame): Traffic Census data with 'objectid', 'ahead_aadt', and 'back_aadt' columns.
    - mapping_dicts (list of dict): Each item includes 'objectid' and potentially zonename_0 to zonename_3.

    Returns:
    - Tuple[np.array, np.array]: Two numpy arrays (StreetLight AADT values, Traffic Census AADT values).
    """
    aadt_non_traditional = []
    aadt_true = []

    for item in mapping_dicts:
        objectid = item["objectid"]
        
        # Determine the correct AADT direction from Traffic Census
        tc_row = df_tc[df_tc["objectid"].astype(str) == objectid]
        if tc_row.empty:
            continue
        
        if int(objectid) % 2 == 0:
            aadt_value = tc_row.iloc[0]["back_aadt"]
        else:
            aadt_value = tc_row.iloc[0]["ahead_aadt"]

        # Try to find a matching STL zonename
        matched = False
        for key in item:
            if key.startswith("zonename_"):
                zonename = item[key]
                stl_row = df_stl[df_stl["zonename"] == zonename]
                if not stl_row.empty:
                    stl_volume = stl_row.iloc[0]["averagedailysegmenttraffic(stlvolume)"]
                    aadt_non_traditional.append(stl_volume)
                    aadt_true.append(aadt_value)
                    matched = True
                    break  # Use the first matching zonename found

        if not matched:
            # Optionally print which objectid failed to match for STL
            print(f"No StreetLight match for objectid {objectid}")

    return np.array(aadt_non_traditional), np.array(aadt_true)

In [9]:
# run the "extract_aadt_comparison_data" function
aadt_non_traditional, aadt_true = extract_aadt_comparison_data(df_stl, df_tc, sr_99_d3_tc_aadt_locations)

KeyError: 'objectid'

In [ ]:
def create_non_trad_df_and_aadt_dict(df_tc, df_stl, interstate_605_aadt_locations):
    # Creating the true_aadt_dict
    true_aadt_dict = {
        str(row["objectid"]): row["back_aadt"] if int(row["objectid"]) % 2 == 0 else row["ahead_aadt"]
        for _, row in df_tc.iterrows()
    }
    
    # Creating non_trad_df
    non_trad_records = []

    for loc in interstate_605_aadt_locations:
        daytype = loc.get("daytype", "0: All Days (M-Su)")
        for key in loc:
            if key.startswith("zonename_"):
                zonename = loc[key]
                stl_match = df_stl[df_stl["zonename"] == zonename]
                if not stl_match.empty:
                    aadt_value = stl_match.iloc[0]["averagedailysegmenttraffic(stlvolume)"]
                    non_trad_records.append({
                        "zonename": zonename,
                        "aadt_value": aadt_value,
                        "daytype": daytype
                    })

    non_trad_df = pd.DataFrame(non_trad_records)

    return true_aadt_dict, non_trad_df

In [ ]:
def compute_tce_confidence_interval_from_df(
    aadt_locations,
    true_aadt_dict,
    non_trad_df,
    confidence=0.95,
    daytype_filter="0: All Days (M-Su)"
):
    """
    Computes the Traffic Count Error (TCE), Confidence Interval, t-critical value, and t-test statistic.
    Only uses non-traditional AADT values with the specified 'daytype'.

    Parameters:
    - aadt_locations (list[dict]): List of location dictionaries.
    - true_aadt_dict (dict): Traditional AADT values keyed by objectid.
    - non_trad_df (DataFrame): DataFrame with non-traditional AADT data.
    - confidence (float): Confidence level for CI calculation.
    - daytype_filter (str): Daytype to filter on (default: "0: All Days (M-Su)")

    Returns:
    - tuple: (mean_tce, ci_lower, ci_upper, t_critical_value, t_test_statistic)
    """
    if true_aadt_dict is None:
        raise ValueError("true_aadt_dict is required but was None.")

    # Filter DataFrame to include only the specified daytype
    filtered_df = non_trad_df[non_trad_df["daytype"] == daytype_filter]

    # Build dict: zonename -> aadt_value
    non_traditional_aadt_dict = dict(zip(
        filtered_df["zonename"],
        filtered_df["aadt_value"]
    ))

    tce_list = []

    for loc in aadt_locations:
        objectid = loc.get("objectid")
        true_aadt = true_aadt_dict.get(objectid)

        if not true_aadt:  # Skip if missing or zero
            continue

        # Sum non-traditional AADT from all zonenames at this location
        non_trad_aadt = sum(
            non_traditional_aadt_dict.get(loc[key], 0)
            for key in loc if key.startswith("zonename_")
        )

        # Compute TCE
        tce = 100 * (non_trad_aadt - true_aadt) / true_aadt
        tce_list.append(tce)

    # If no valid TCEs were computed
    if not tce_list:
        return None, None, None, None, None

    # Convert to NumPy array and calculate statistics
    tce_array = np.array(tce_list)
    mean_tce = np.mean(tce_array)
    std_tce = np.std(tce_array, ddof=1)
    n = len(tce_array)
    se = std_tce / np.sqrt(n)
    
    # Compute the t-critical value
    df = n - 1
    t_critical = stats.t.ppf((1 + confidence) / 2, df)
    
    # Calculate the Confidence Interval
    ci_lower = mean_tce - t_critical * se
    ci_upper = mean_tce + t_critical * se
    
    # Compute the t-test statistic
    t_test_statistic = mean_tce / se if se != 0 else None

    return mean_tce, ci_lower, ci_upper, t_critical, t_test_statistic

In [ ]:
true_aadt_dict, non_trad_df = create_non_trad_df_and_aadt_dict(df_tc, df_stl, interstate_99_aadt_locations)

In [ ]:
# LOC = interstate_99_aadt_locations

In [ ]:
mean_tce, ci_lower, ci_upper, t_critical, t_test_statistic = compute_tce_confidence_interval_from_df(
    aadt_locations=LOC,            # kept for signature compatibility
    true_aadt_dict=true_aadt_dict, # keys: "<location_description>|ahead/back"
    non_trad_df=non_trad_df,
    confidence=0.95,
    daytype_filter="0: All Days (M-Su)"
)

In [ ]:
# Print the results
print("Mean TCE:", mean_tce)
print("95% Confidence Interval:", (ci_lower, ci_upper))
print("t-critical:", t_critical)
print("t-test statistic:", t_test_statistic)

### Mean TCE: -3.62
Traffic Census Error (TCE)
* 

### 95% Confidence Interval (-10.78%, 3.54%)
* 

### T-Test Statistic  
* 
### Summary
* 
